In [1]:
pip install requests beautifulsoup4 pandas lxml openpyxl

Note: you may need to restart the kernel to use updated packages.


In [20]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import re

CIK = "0001067983"
BASE = "https://www.sec.gov"
DATA_URL = f"https://data.sec.gov/submissions/CIK{CIK}.json"
HEADERS = {
    "User-Agent": "Mozilla/5.0 (DataResearchBot/1.0; your_email@example.com)"
}

session = requests.Session()
session.headers.update(HEADERS)

# ==========================
# 1. FETCH JSON INDEX
# ==========================
def fetch_filing_json():
    print("1️⃣ Fetching SEC JSON index...")
    r = session.get(DATA_URL)
    r.raise_for_status()
    return r.json()

# ==========================
# 2. EXTRACT FIRST 15 VALID 13F-HR (NOT 13F-HR/A)
# ==========================
def extract_13f_filings(data, limit=15):
    print("2️⃣ Filtering first 15 valid 13F-HR filings (excluding amendments)...")
    recent = data["filings"]["recent"]
    filings = []
    
    for form, acc, date in zip(
        recent["form"],
        recent["accessionNumber"],
        recent["filingDate"]
    ):
        # Only accept "13F-HR" exactly, NOT "13F-HR/A"
        if form == "13F-HR":
            filings.append((acc, date))
            print(f"   ✓ {form} | {date} | {acc}")
            if len(filings) == limit:
                break
    
    print(f"\n   Found {len(filings)} filings")
    return filings

# ==========================
# 3. GET INFORMATION TABLE HTML DOCUMENT LINK
# ==========================
def get_information_table_url(accession):
    """
    Navigate to the filing's document page and find the INFORMATION TABLE .html file
    ONLY .html files - NEVER .xml
    """
    # Remove dashes for the URL path
    acc_no_dash = accession.replace("-", "")
    
    # Build the filing index page URL
    filing_page = f"{BASE}/Archives/edgar/data/{int(CIK)}/{acc_no_dash}/{accession}-index.htm"
    
    print(f"   📄 Fetching: {filing_page}")
    
    try:
        r = session.get(filing_page)
        r.raise_for_status()
        soup = BeautifulSoup(r.text, 'html.parser')
        
        # Find the document table
        table = soup.find('table', {'class': 'tableFile'})
        if not table:
            # Try alternate table structure
            table = soup.find('table', summary='Document Format Files')
        
        if not table:
            print("   ⚠️ Document table not found")
            return None
        
        print("   🔍 Searching for INFORMATION TABLE .html document...")
        print("   🐛 DEBUG: Showing ALL links in ALL rows:")
        
        # Get all rows
        all_rows = table.find_all('tr')
        
        for idx, row in enumerate(all_rows[1:], start=1):  # Skip header row
            cells = row.find_all('td')
            
            if len(cells) >= 4:
                # Get the ACTUAL TEXT from the document cell (column 2)
                doc_cell_text = cells[2].get_text(strip=True)
                
                # Get type from column 3
                doc_type = cells[3].get_text(strip=True)
                
                # Get ALL links in this cell
                all_links = cells[2].find_all('a')
                
                print(f"      Row {idx}: Type='{doc_type}', Text='{doc_cell_text}'")
                for link in all_links:
                    href = link.get('href')
                    link_text = link.get_text(strip=True)
                    print(f"         → Link text: '{link_text}', href: '{href}'")
                
                # Check if this is INFORMATION TABLE and the text is .html
                if "INFORMATION TABLE" in doc_type.upper() and doc_cell_text.endswith('.html'):
                    # Find the link that actually points to the .html file
                    for link in all_links:
                        href = link.get('href')
                        link_text = link.get_text(strip=True)
                        
                        # Match the link text with our target filename
                        if link_text == doc_cell_text:
                            full_url = BASE + href if href.startswith('/') else href
                            
                            print(f"\n   ✅ Found INFORMATION TABLE HTML: {doc_cell_text}")
                            print(f"   🔗 URL: {full_url}")
                            return full_url
        
        print("\n   ⚠️ No INFORMATION TABLE .html file found")
        return None
        
    except Exception as e:
        print(f"   ❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

# ==========================
# 4. SCRAPE TABLE
# ==========================
def scrape_html_table(url):
    print(f"   📊 Scraping table from INFORMATION TABLE HTML...")
    try:
        # Use the session with proper headers
        r = session.get(url)
        r.raise_for_status()
        
        # Try pandas read_html with the HTML content
        tables = pd.read_html(r.text)
        
        if tables:
            # Find the table with the most rows (likely the holdings table)
            main_table = max(tables, key=lambda x: len(x))
            print(f"   ✅ Extracted table with {len(main_table)} rows and {len(main_table.columns)} columns")
            
            # Show first few column names for verification
            if len(main_table.columns) > 0:
                print(f"   📋 Columns: {list(main_table.columns[:5])}")
            
            return main_table
        
        print("   ⚠️ No tables found with pandas")
        return None
            
    except Exception as e:
        print(f"   ❌ Error scraping: {e}")
        import traceback
        traceback.print_exc()
        return None

# ==========================
# 5. MAIN PIPELINE
# ==========================
def main():
    try:
        data = fetch_filing_json()
        filings = extract_13f_filings(data, limit=15)
        
        if not filings:
            print("❌ No filings found!")
            return
        
        # Collect all successful dataframes first
        all_data = []
        
        for i, (accession, filing_date) in enumerate(filings, 1):
            print(f"\n{'='*60}")
            print(f"Processing {i}/{len(filings)} — Filing Date: {filing_date}")
            print(f"Accession: {accession}")
            print('='*60)
            
            # Get the INFORMATION TABLE HTML URL
            html_url = get_information_table_url(accession)
            
            if not html_url:
                print("   ⚠️ Skipping — No INFORMATION TABLE .html found")
                continue
            
            # Scrape the table data
            df = scrape_html_table(html_url)
            
            if df is not None and not df.empty:
                # Create sheet name from filing date
                sheet = filing_date.replace("-", "_")[:31]
                all_data.append((sheet, df))
                print(f"   ✅ Data ready for sheet: {sheet}")
            else:
                print("   ⚠️ Skipping — No data extracted")
            
            # Be respectful to SEC servers
            time.sleep(0.5)
        
        # Now write to Excel only if we have data
        if all_data:
            print(f"\n{'='*60}")
            print(f"📝 Writing {len(all_data)} sheets to Excel...")
            print('='*60)
            
            with pd.ExcelWriter("13F_Data.xlsx", engine="openpyxl") as writer:
                for sheet_name, df in all_data:
                    df.to_excel(writer, sheet_name=sheet_name, index=False)
                    print(f"   ✓ Written sheet: {sheet_name}")
            
            print(f"\n{'='*60}")
            print(f"✅ SUCCESS — Processed {len(all_data)}/{len(filings)} filings")
            print(f"📁 Excel file created: 13F_Data.xlsx")
            print('='*60)
        else:
            print(f"\n{'='*60}")
            print("❌ No data was extracted from any filing")
            print("Excel file NOT created")
            print('='*60)
    
    except Exception as e:
        print(f"\n❌ FATAL ERROR: {e}")
        import traceback
        traceback.print_exc()

# ==========================
# RUN
# ==========================
if __name__ == "__main__":
    main()

1️⃣ Fetching SEC JSON index...
2️⃣ Filtering first 15 valid 13F-HR filings (excluding amendments)...
   ✓ 13F-HR | 2025-11-14 | 0001193125-25-282901
   ✓ 13F-HR | 2025-08-14 | 0000950123-25-008343
   ✓ 13F-HR | 2025-05-15 | 0000950123-25-005701
   ✓ 13F-HR | 2025-02-14 | 0000950123-25-002701
   ✓ 13F-HR | 2024-11-14 | 0000950123-24-011775
   ✓ 13F-HR | 2024-08-14 | 0000950123-24-008740
   ✓ 13F-HR | 2024-05-15 | 0000950123-24-005622
   ✓ 13F-HR | 2024-02-14 | 0000950123-24-002518
   ✓ 13F-HR | 2023-11-14 | 0000950123-23-010898
   ✓ 13F-HR | 2023-08-14 | 0000950123-23-008074
   ✓ 13F-HR | 2023-05-15 | 0000950123-23-005270
   ✓ 13F-HR | 2023-02-14 | 0000950123-23-002585
   ✓ 13F-HR | 2022-11-14 | 0000950123-22-012275
   ✓ 13F-HR | 2022-08-15 | 0000950123-22-009450
   ✓ 13F-HR | 2022-05-16 | 0000950123-22-006442

   Found 15 filings

Processing 1/15 — Filing Date: 2025-11-14
Accession: 0001193125-25-282901
   📄 Fetching: https://www.sec.gov/Archives/edgar/data/1067983/000119312525282901/0

/var/folders/21/13dv45411jx7lncnks5rdpqm0000gn/T/ipykernel_19495/1807052526.py:140: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)


   ✅ Extracted table with 118 rows and 13 columns
   📋 Columns: [0, 1, 2, 3, 4]
   ✅ Data ready for sheet: 2025_11_14

Processing 2/15 — Filing Date: 2025-08-14
Accession: 0000950123-25-008343
   📄 Fetching: https://www.sec.gov/Archives/edgar/data/1067983/000095012325008343/0000950123-25-008343-index.htm
   🔍 Searching for INFORMATION TABLE .html document...
   🐛 DEBUG: Showing ALL links in ALL rows:
      Row 1: Type='13F-HR', Text='primary_doc.html'
         → Link text: 'primary_doc.html', href: '/Archives/edgar/data/1067983/000095012325008343/xslForm13F_X02/primary_doc.xml'
      Row 2: Type='13F-HR', Text='primary_doc.xml'
         → Link text: 'primary_doc.xml', href: '/Archives/edgar/data/1067983/000095012325008343/primary_doc.xml'
      Row 3: Type='INFORMATION TABLE', Text='43977.html'
         → Link text: '43977.html', href: '/Archives/edgar/data/1067983/000095012325008343/xslForm13F_X02/43977.xml'

   ✅ Found INFORMATION TABLE HTML: 43977.html
   🔗 URL: https://www.sec.gov/

/var/folders/21/13dv45411jx7lncnks5rdpqm0000gn/T/ipykernel_19495/1807052526.py:140: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)



Processing 3/15 — Filing Date: 2025-05-15
Accession: 0000950123-25-005701
   📄 Fetching: https://www.sec.gov/Archives/edgar/data/1067983/000095012325005701/0000950123-25-005701-index.htm
   🔍 Searching for INFORMATION TABLE .html document...
   🐛 DEBUG: Showing ALL links in ALL rows:
      Row 1: Type='13F-HR', Text='primary_doc.html'
         → Link text: 'primary_doc.html', href: '/Archives/edgar/data/1067983/000095012325005701/xslForm13F_X02/primary_doc.xml'
      Row 2: Type='13F-HR', Text='primary_doc.xml'
         → Link text: 'primary_doc.xml', href: '/Archives/edgar/data/1067983/000095012325005701/primary_doc.xml'
      Row 3: Type='INFORMATION TABLE', Text='form13fInfoTable.html'
         → Link text: 'form13fInfoTable.html', href: '/Archives/edgar/data/1067983/000095012325005701/xslForm13F_X02/form13fInfoTable.xml'

   ✅ Found INFORMATION TABLE HTML: form13fInfoTable.html
   🔗 URL: https://www.sec.gov/Archives/edgar/data/1067983/000095012325005701/xslForm13F_X02/form13fInfoT

...


Processing 14/15 — Filing Date: 2022-08-15
Accession: 0000950123-22-009450
   📄 Fetching: https://www.sec.gov/Archives/edgar/data/1067983/000095012322009450/0000950123-22-009450-index.htm
   🔍 Searching for INFORMATION TABLE .html document...
   🐛 DEBUG: Showing ALL links in ALL rows:
      Row 1: Type='13F-HR', Text='primary_doc.html'
         → Link text: 'primary_doc.html', href: '/Archives/edgar/data/1067983/000095012322009450/xslForm13F_X01/primary_doc.xml'
      Row 2: Type='13F-HR', Text='primary_doc.xml'
         → Link text: 'primary_doc.xml', href: '/Archives/edgar/data/1067983/000095012322009450/primary_doc.xml'
      Row 3: Type='INFORMATION TABLE', Text='15796.html'
         → Link text: '15796.html', href: '/Archives/edgar/data/1067983/000095012322009450/xslForm13F_X01/15796.xml'

   ✅ Found INFORMATION TABLE HTML: 15796.html
   🔗 URL: https://www.sec.gov/Archives/edgar/data/1067983/000095012322009450/xslForm13F_X01/15796.xml
   📊 Scraping table from INFORMATION TABLE HT

/var/folders/21/13dv45411jx7lncnks5rdpqm0000gn/T/ipykernel_19495/1807052526.py:140: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)



Processing 15/15 — Filing Date: 2022-05-16
Accession: 0000950123-22-006442
   📄 Fetching: https://www.sec.gov/Archives/edgar/data/1067983/000095012322006442/0000950123-22-006442-index.htm
   🔍 Searching for INFORMATION TABLE .html document...
   🐛 DEBUG: Showing ALL links in ALL rows:
      Row 1: Type='13F-HR', Text='primary_doc.html'
         → Link text: 'primary_doc.html', href: '/Archives/edgar/data/1067983/000095012322006442/xslForm13F_X01/primary_doc.xml'
      Row 2: Type='13F-HR', Text='primary_doc.xml'
         → Link text: 'primary_doc.xml', href: '/Archives/edgar/data/1067983/000095012322006442/primary_doc.xml'
      Row 3: Type='INFORMATION TABLE', Text='13095.html'
         → Link text: '13095.html', href: '/Archives/edgar/data/1067983/000095012322006442/xslForm13F_X01/13095.xml'

   ✅ Found INFORMATION TABLE HTML: 13095.html
   🔗 URL: https://www.sec.gov/Archives/edgar/data/1067983/000095012322006442/xslForm13F_X01/13095.xml
   📊 Scraping table from INFORMATION TABLE HT

/var/folders/21/13dv45411jx7lncnks5rdpqm0000gn/T/ipykernel_19495/1807052526.py:140: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables = pd.read_html(r.text)



📝 Writing 15 sheets to Excel...
   ✓ Written sheet: 2025_11_14
   ✓ Written sheet: 2025_08_14
   ✓ Written sheet: 2025_05_15
   ✓ Written sheet: 2025_02_14
   ✓ Written sheet: 2024_11_14
   ✓ Written sheet: 2024_08_14
   ✓ Written sheet: 2024_05_15
   ✓ Written sheet: 2024_02_14
   ✓ Written sheet: 2023_11_14
   ✓ Written sheet: 2023_08_14
   ✓ Written sheet: 2023_05_15
   ✓ Written sheet: 2023_02_14
   ✓ Written sheet: 2022_11_14
   ✓ Written sheet: 2022_08_15
   ✓ Written sheet: 2022_05_16

✅ SUCCESS — Processed 15/15 filings
📁 Excel file created: 13F_Data.xlsx
